<a href="https://colab.research.google.com/github/bt-ktm/QOSF-Mentorship-Task-3/blob/main/Binayek_Task3_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


In [4]:
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 104.1 MB/s eta 0:00:00


In [5]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
import random

In [21]:
# 1. NOISE MODEL
# ============================================================================

def noise_model(a, b, circuit):
    """
    Given a circuit, adds Pauli X with probability 'a' and Pauli Z with
    probability 'b' to each qubit after each gate layer.

    Args:
        a: Probability of X error
        b: Probability of Z error
        circuit: Input quantum circuit

    Returns:
        circuit_with_noise: Circuit with noise added
    """
    noisy_circuit = QuantumCircuit(circuit.num_qubits, circuit.num_clbits)

    # Copy the original circuit and add noise after each instruction
    for instruction in circuit.data:
        noisy_circuit.append(instruction)

        # Add noise to each qubit involved in the instruction
        qubits = instruction.qubits
        for qubit in qubits:
            qubit_idx = circuit.qubits.index(qubit)

            # Apply X error with probability a
            if random.random() < a:
                noisy_circuit.x(qubit_idx)

            # Apply Z error with probability b
            if random.random() < b:
                noisy_circuit.z(qubit_idx)

    return noisy_circuit

In [22]:
def test_noise_model():
    """Test the noise model with simple circuits"""
    print("=" * 70)
    print("TESTING NOISE MODEL")
    print("=" * 70)

    # Test 1: Simple X gate
    print("\nTest 1: Single X gate with noise (a=0.3, b=0.2)")
    qc = QuantumCircuit(1, 1)
    qc.x(0)
    qc.measure(0, 0)

    simulator = AerSimulator()

    # Run without noise
    result = simulator.run(qc, shots=1000).result()
    counts = result.get_counts()
    print(f"Without noise: {counts}")

    # Run with noise multiple times to see distribution
    noisy_counts = {}
    for _ in range(100):
        noisy_qc = noise_model(0.3, 0.2, qc)
        result = simulator.run(noisy_qc, shots=10).result()
        c = result.get_counts()
        for key in c:
            noisy_counts[key] = noisy_counts.get(key, 0) + c[key]

    print(f"With noise (aggregated): {noisy_counts}")

    # Test 2: Hadamard gate
    print("\nTest 2: Hadamard gate with noise (a=0.2, b=0.2)")
    qc2 = QuantumCircuit(1, 1)
    qc2.h(0)
    qc2.measure(0, 0)

    result = simulator.run(qc2, shots=1000).result()
    print(f"Without noise: {result.get_counts()}")

    noisy_counts = {}
    for _ in range(100):
        noisy_qc = noise_model(0.2, 0.2, qc2)
        result = simulator.run(noisy_qc, shots=10).result()
        c = result.get_counts()
        for key in c:
            noisy_counts[key] = noisy_counts.get(key, 0) + c[key]

    print(f"With noise (aggregated): {noisy_counts}")


In [23]:
# ============================================================================
# 2. QUANTUM REPETITION CODE (3-qubit for X errors)
# ============================================================================

def encode_repetition(qc, data_qubit, ancilla1, ancilla2):
    """Encode one logical qubit into 3 physical qubits using repetition code"""
    qc.cx(data_qubit, ancilla1)
    qc.cx(data_qubit, ancilla2)


def decode_repetition(qc, q0, q1, q2, syndrome1, syndrome2):
    """
    Decode and correct errors in repetition code
    Measures syndromes and corrects based on majority vote
    """
    # Measure parity checks
    qc.cx(q0, syndrome1)
    qc.cx(q1, syndrome1)
    qc.cx(q1, syndrome2)
    qc.cx(q2, syndrome2)
    qc.measure([syndrome1, syndrome2], [0, 1])

    # Based on syndrome, apply corrections
    # This is simplified - in practice would use classical feedback
    qc.barrier()


def test_repetition_code():
    """Test repetition code with X errors only"""
    print("\n" + "=" * 70)
    print("TESTING QUANTUM REPETITION CODE")
    print("=" * 70)

    simulator = AerSimulator()
    shots = 1000

In [24]:
def test_repetition_code():
    """Test repetition code with X errors only"""
    print("\n" + "=" * 70)
    print("TESTING QUANTUM REPETITION CODE")
    print("=" * 70)

    simulator = AerSimulator()
    shots = 1000

    # Test encoding |1⟩ state
    print("\nTest: Encoding |1⟩ with X errors (a=0.15, b=0.0)")

    qc = QuantumCircuit(5, 3)  # 3 data qubits, 2 syndrome qubits, 3 classical bits

    # Prepare |1⟩ state
    qc.x(0)
    qc.barrier()

    # Encode
    encode_repetition(qc, 0, 1, 2)
    qc.barrier()

    # Add noise (X errors only)
    noisy_qc = noise_model(0.15, 0.0, qc)

    # Decode with syndrome measurement
    decode_repetition(noisy_qc, 0, 1, 2, 3, 4)

    # Final measurement of logical qubit
    noisy_qc.measure(0, 2)

    result = simulator.run(noisy_qc, shots=shots).result()
    counts = result.get_counts()
    print(f"Results: {counts}")

    # Calculate error rate
    errors = sum(count for state, count in counts.items() if state[-1] == '0')
    print(f"Error rate: {errors/shots:.2%}")

    print("\nWhy repetition code fails for Z errors:")
    print("- Repetition code encodes: |0⟩ → |000⟩ and |1⟩ → |111⟩")
    print("- X errors flip bits: |000⟩ → |001⟩, detectable by parity checks")
    print("- Z errors add phase: |+⟩ → |-⟩, but don't change computational basis")
    print("- Syndrome measurements collapse superpositions, destroying phase info")
    print("- Need different encoding strategy (like Shor code) for Z errors")


In [25]:

# ============================================================================
# 3. SHOR CODE (9-qubit)
# ============================================================================

def encode_shor(qc, data_qubit):
    """
    Encode one logical qubit into 9 physical qubits using Shor code
    Protects against both X and Z errors
    """
    # First level: bit-flip code (protect against X)
    qc.cx(data_qubit, 3)
    qc.cx(data_qubit, 6)

    # Second level: phase-flip code (protect against Z)
    for i in [0, 3, 6]:
        qc.h(i)
        qc.cx(i, i+1)
        qc.cx(i, i+2)


def syndrome_measurement_shor(qc):
    """
    Measure syndromes for Shor code
    Requires 8 ancilla qubits for syndrome extraction
    """
    # This is a simplified version
    # X-error syndromes (within each block of 3)
    for block in [0, 3, 6]:
        qc.cx(block, 9)
        qc.cx(block+1, 9)
        qc.cx(block+1, 10)
        qc.cx(block+2, 10)

    # Z-error syndromes (between blocks)
    for i in [0, 3]:
        qc.h(i)
        qc.h(i+3)
        qc.cx(i, 11)
        qc.cx(i+3, 11)

In [26]:
def test_shor_code():
    """Test Shor code with both X and Z errors"""
    print("\n" + "=" * 70)
    print("TESTING SHOR CODE")
    print("=" * 70)

    simulator = AerSimulator()
    shots = 1000

    print("\nTest: Encoding |1⟩ with X and Z errors (a=0.1, b=0.1)")

    qc = QuantumCircuit(9, 1)

    # Prepare |1⟩ state
    qc.x(0)
    qc.barrier()

    # Encode with Shor code
    encode_shor(qc, 0)
    qc.barrier()

    # Add noise
    noisy_qc = noise_model(0.1, 0.1, qc)
    noisy_qc.barrier()

    # Decode (simplified - just measure first qubit)
    # Full decoding would require syndrome measurement and correction
    noisy_qc.measure(0, 0)

    result = simulator.run(noisy_qc, shots=shots).result()
    counts = result.get_counts()
    print(f"Results: {counts}")

    # Calculate error rate
    errors = sum(count for state, count in counts.items() if state == '0')
    print(f"Error rate: {errors/shots:.2%}")

    print("\nShor code properties:")
    print("- Uses 9 qubits to encode 1 logical qubit")
    print("- Combines bit-flip and phase-flip codes")
    print("- Can correct 1 arbitrary single-qubit error")
    print("- Protects against both X and Z errors")


In [27]:
# ============================================================================
# 4. HAMMING [7,4,3] CODE
# ============================================================================

def encode_hamming_743(qc):
    """
    Encode 4 logical qubits into 7 physical qubits using Hamming [7,4,3] code
    This is a CSS code that can correct 1 bit-flip error
    """
    # Qubits 0-3 are data qubits, 4-6 are parity qubits

    # Parity qubit 4: checks qubits 0, 1, 3
    qc.cx(0, 4)
    qc.cx(1, 4)
    qc.cx(3, 4)

    # Parity qubit 5: checks qubits 0, 2, 3
    qc.cx(0, 5)
    qc.cx(2, 5)
    qc.cx(3, 5)

    # Parity qubit 6: checks qubits 1, 2, 3
    qc.cx(1, 6)
    qc.cx(2, 6)
    qc.cx(3, 6)


def syndrome_measurement_hamming(qc):
    """Measure syndromes for Hamming code"""
    # Measure parity qubits
    qc.measure([4, 5, 6], [0, 1, 2])


In [28]:
def test_hamming_code():
    """Test Hamming [7,4,3] code"""
    print("\n" + "=" * 70)
    print("TESTING HAMMING [7,4,3] CODE")
    print("=" * 70)

    simulator = AerSimulator()
    shots = 1000

    print("\nTest: Encoding with X and Z errors (a=0.08, b=0.08)")

    qc = QuantumCircuit(7, 3)

    # Prepare some data in qubits 0-3
    qc.x(0)
    qc.x(2)
    qc.barrier()

    # Encode
    encode_hamming_743(qc)
    qc.barrier()

    # Add noise
    noisy_qc = noise_model(0.08, 0.08, qc)
    noisy_qc.barrier()

    # Measure syndromes
    syndrome_measurement_hamming(noisy_qc)

    result = simulator.run(noisy_qc, shots=shots).result()
    counts = result.get_counts()
    print(f"Syndrome measurements: {counts}")

    print("\nHamming code properties:")
    print("- Encodes 4 logical qubits into 7 physical qubits")
    print("- Can detect and correct 1 bit-flip error")
    print("- More efficient than Shor code (better rate: 4/7 vs 1/9)")
    print("- Classical Hamming code adapted to quantum setting")


In [29]:
# ============================================================================
# ANALYSIS
# ============================================================================

def print_analysis():
    """Print comparison and challenges"""
    print("\n" + "=" * 70)
    print("ANALYSIS: SHOR VS HAMMING CODES")
    print("=" * 70)

    print("\nKey Differences:")
    print("\n1. Encoding Rate:")
    print("   - Shor: 1/9 (1 logical → 9 physical qubits)")
    print("   - Hamming: 4/7 (4 logical → 7 physical qubits)")
    print("   → Hamming is more efficient")

    print("\n2. Error Correction Capability:")
    print("   - Shor: Corrects arbitrary single-qubit errors (X, Y, Z)")
    print("   - Hamming: Primarily corrects bit-flip (X) errors")
    print("   → Shor provides fuller protection")

    print("\n3. Structure:")
    print("   - Shor: Concatenated code (bit-flip + phase-flip)")
    print("   - Hamming: CSS code (classical code adapted)")

    print("\n4. Historical Significance:")
    print("   - Shor: First quantum error correction code (1995)")
    print("   - Hamming: Classical code (1950s) adapted to quantum")

    print("\n" + "=" * 70)
    print("CHALLENGES IN BUILDING ERROR-CORRECTING CODES")
    print("=" * 70)

    print("\n1. Syndrome Measurement Without Collapse:")
    print("   - Must extract error information without destroying quantum state")
    print("   - Requires entangling data qubits with ancillas carefully")

    print("\n2. Overhead:")
    print("   - Need many physical qubits to encode few logical qubits")
    print("   - Current hardware has limited qubit counts")

    print("\n3. Noise in Error Correction Itself:")
    print("   - Syndrome measurements and corrections introduce new errors")
    print("   - Need error rates below threshold (~1% for surface codes)")

    print("\n4. Real-time Classical Processing:")
    print("   - Must decode syndromes and apply corrections quickly")
    print("   - Decoherence times are short (~μs to ms)")

    print("\n5. Circuit Depth:")
    print("   - Error correction adds many gates")
    print("   - Each gate is a potential error source")
    print("   - Trade-off between protection and gate errors")

    print("\n6. Fault-Tolerance:")
    print("   - Errors in error correction can propagate")
    print("   - Need fault-tolerant protocols (beyond scope here)")

    print("\n7. Different Error Types:")
    print("   - X, Z errors behave differently")
    print("   - Need codes that handle both (like Shor)")
    print("   - Correlated errors even harder")

    print("\n8. Implementation Complexity:")
    print("   - Require many controlled operations")
    print("   - Not all gate types equally available on hardware")
    print("   - Connectivity constraints on real devices")


In [30]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Set random seed for reproducibility
    random.seed(42)
    np.random.seed(42)

    # Run all tests
    test_noise_model()
    test_repetition_code()
    test_shor_code()
    test_hamming_code()
    print_analysis()

    print("\n" + "=" * 70)
    print("All tests completed!")
    print("=" * 70)

TESTING NOISE MODEL

Test 1: Single X gate with noise (a=0.3, b=0.2)
Without noise: {'1': 1000}
With noise (aggregated): {'1': 720, '0': 280}

Test 2: Hadamard gate with noise (a=0.2, b=0.2)
Without noise: {'0': 503, '1': 497}
With noise (aggregated): {'1': 503, '0': 497}

TESTING QUANTUM REPETITION CODE

Test: Encoding |1⟩ with X errors (a=0.15, b=0.0)
Results: {'101': 1000}
Error rate: 0.00%

Why repetition code fails for Z errors:
- Repetition code encodes: |0⟩ → |000⟩ and |1⟩ → |111⟩
- X errors flip bits: |000⟩ → |001⟩, detectable by parity checks
- Z errors add phase: |+⟩ → |-⟩, but don't change computational basis
- Syndrome measurements collapse superpositions, destroying phase info
- Need different encoding strategy (like Shor code) for Z errors

TESTING SHOR CODE

Test: Encoding |1⟩ with X and Z errors (a=0.1, b=0.1)
Results: {'0': 478, '1': 522}
Error rate: 47.80%

Shor code properties:
- Uses 9 qubits to encode 1 logical qubit
- Combines bit-flip and phase-flip codes
- Can c